In [ ]:
import numpy as np
import pandas as pd
lottorydf = pd.read_csv('lottory.csv')
lottorydf=lottorydf.sort_values(by=['期號'],ascending=True)
lottorydf=lottorydf.reset_index(drop=True)
lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6','預測2區']] = lottorydf[['前1','前2','前3','前4','前5','前6','后區']].shift(-1)
print(lottorydf.tail())
print(lottorydf.shape)

In [ ]:
# print(lottorydf.iloc[:,8:14]/38)
lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6']]=lottorydf[['預測1','預測2','預測3','預測4','預測5','預測6']]/38
lottorydf[['前1','前2','前3','前4','前5','前6']]=lottorydf[['前1','前2','前3','前4','前5','前6']]/38
lottorydf[['后區','預測2區']]=lottorydf[['后區','預測2區']]/8
print(lottorydf.tail())

In [ ]:
def preparedata(df, ref_day, predict_day):
    X_train, Y_train = [], []
    for i in range(df.shape[0]-predict_day-ref_day+1):
        X_train.append(np.array(df.iloc[i:i+ref_day,1:8]))
        Y_train.append(np.array(df.iloc[i+ref_day:i+ref_day+predict_day,8:15]))
    return np.array(X_train), np.array(Y_train)

In [ ]:
X_train,Y_train= preparedata(lottorydf[:lottorydf.shape[0]-18],8,1)
X_train=X_train.transpose(0,2,1)
Y_train=Y_train.transpose(0,2,1)
print(X_train.shape,len(X_train),Y_train.shape,len(Y_train),X_train.ndim)
X_test,Y_test = preparedata(lottorydf[lottorydf.shape[0]-18:],8,1)
X_test=X_test.transpose(0,2,1)
Y_test=Y_test.transpose(0,2,1)
print(X_test.shape,len(X_test),Y_test.shape,len(Y_test),X_test.ndim)

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM,Dense,Activation
import tensorflow as tf

In [ ]:
model = Sequential()
model.add(LSTM(49,input_shape=(7,8),activation='relu'))#, return_sequences=True, stateful=True))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'mean_squared_error',metrics = ['accuracy'])
model.summary()

In [ ]:
model.fit(X_train,Y_train, epochs=10,batch_size=1)


In [ ]:
model.evaluate(X_test,Y_test,batch_size=1)
predicts = model.predict(X_test,batch_size=1)
print(type(predicts))
print(predicts)
# print(model.predict_classes(X_test,batch_size=1))

In [ ]:
print(predicts.shape)
a = predicts.transpose(0,2,1)
new= a.squeeze()
new = pd.DataFrame(new)
new[[0,1,2,3,4,5]] = new[[0,1,2,3,4,5]]*38
new[[6]]=new[[6]]*8
new = new.astype(int)
new